In [ ]:
import sys
import pandas as pd
from load_data import get_data, load_pkl, save_pkl
from processing_helpers import get_cue_segments, get_power_spec, get_peak_fits
from tally_helper import TallyWindow
# from plot_window import plot_processing
import pickle as pkl
import numpy as np
import DataProcessingTools as DPT
import os
import matplotlib.pyplot as plt
from fooof import FOOOF
from ipywidgets import widgets, HBox, VBox, Output
from scipy.stats import sem, f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

%matplotlib qt


In [ ]:
# Global variables to store the session data
lfp_df = pd.DataFrame()
ch_num_list = []
lfp_mne = []
session_start_time = 0
markers = []
timeStamps = []
sampling_frequency = 0
segments_df = pd.DataFrame()

In [ ]:
def load_data(day):
    """
    Load LFP data for a given day.
    """
    global lfp_df, ch_num_list, lfp_mne, session_start_time, markers, timeStamps, sampling_frequency
    print(f"Loading data for {day}...")
    lfp_df, ch_num_list, lfp_mne, session_start_time, markers, timeStamps, sampling_frequency = get_data(day)

    if lfp_df is not None:
        print(f"Loaded {len(ch_num_list)} channels.")
        print(lfp_df.head())
    else:
        print("Failed to load data.")

def get_segment(segments_df, window_size, method='welch', max_n_peaks=3, peak_threshold=2, peak_fit_range=[1,12]):
    """
    Extract segments from the LFP data using the specified window size.
    """
    if lfp_df.empty or not markers or not timeStamps:
        print("Please load data before extracting segments.")
        return

    segments_df = get_cue_segments(lfp_df, markers, timeStamps, window_size=window_size)
    print(f"Extracted {len(segments_df)} segments.")

    frequencies = []
    psd_list = []
    p_stds_l = []
    p_means_l = []
    peak_gauss_l = []
    flat_spec_l = []
    flat_spec_og_l = []
    ap_fit_og_l = []
    peaks_l, peak_freqs_l = [], []

    count = 0
    segments_list = segments_df['segment'].values  # Convert df to array for faster processing

    for segment in segments_list:
        freq, psd = get_power_spec(segment, sampling_frequency, method=method)
        frequencies.append(freq)


        psd, ap_fit_og, flat_spec_og, p_stds, p_means, peak_gauss, flat_specs, peaks, peak_freqs= get_peak_fits(psd, freq, peak_fit_range, max_n_peaks=max_n_peaks, peak_threshold=peak_threshold)
        psd_list.append(psd)
        p_stds_l.append(p_stds)
        p_means_l.append(p_means)
        peak_gauss_l.append(peak_gauss)
        flat_spec_l.append(flat_specs)
        flat_spec_og_l.append(flat_spec_og)
        ap_fit_og_l.append(ap_fit_og)
        peaks_l.append(peaks)
        peak_freqs_l.append(peak_freqs)

        count += 1
        print(f"Processed segment {count}")

    segments_df['psd'] = psd_list
    segments_df['freq'] = frequencies
    segments_df['peak_stds'] = p_stds_l
    segments_df['peak_means'] = p_means_l
    segments_df['peak_gauss'] = peak_gauss_l
    segments_df['flat_specs'] = flat_spec_l
    segments_df['flat_spec_og'] = flat_spec_og_l
    segments_df['ap_fit_og'] = ap_fit_og_l
    segments_df['peaks'] = peaks_l
    segments_df['peak_freqs'] = peak_freqs_l
    print("Segments processed successfully.")

def show_tally():
    """
    Display tally information for the segments DataFrame.
    """
    if segments_df.empty:
        print("Please generate segments before viewing the tally.")
        return

    # Example of tally logic; customize as needed
    print("Tally of segments by channel:")
    print(segments_df.groupby('channel').size())
    TallyWindow(segments_df, ch_num_list).show()


In [ ]:
# Load Raw Data
day = "20181105"
load_data(day)

# Extract segments with a specific window size and process
window_size = 1000  # e.g., 1000 ms
peak_fit_range = [1,14]
get_segment(window_size, method='welch', max_n_peaks=3, peak_threshold=1.76, peak_fit_range=peak_fit_range)

save_pkl("control_segments.pkl")